C Conversion
---
All we need to do to convert our Mahalanobis Distance model to C is to save the mean vector and inverse of the covariance matrix as constant arrays.

In [17]:
from os import listdir
from os.path import join
import numpy as np
import scipy as sp
from scipy import stats
import c_writer

In [19]:
# Settings (define file locations)
models_path = 'models'  # Where we can find the model files (relative path location)
md_file_name = 'md_model-steady'   # Mahalanobis Distance model arrays (.npz will be added)
c_model_name = 'md_model-steady'   # Will be given .h suffix

c_mean_name = 'model_mu'
c_inv_cov_name = 'model_inv_cov'

In [20]:
# Load model
with np.load(join(models_path, md_file_name) + '.npz') as data:
    model_mu = data['model_mu']
    model_cov = data['model_cov']
print(model_mu)
print(model_cov)

[0.00067966 0.00066138 0.00080888]
[[5.13535666e-09 6.07580327e-10 2.69495019e-11]
 [6.07580327e-10 5.05294802e-09 1.30720871e-09]
 [2.69495019e-11 1.30720871e-09 1.10838602e-08]]


In [21]:
# Calculate inverse of covariance matrix (as it's constant)
inv_cov = sp.linalg.inv(model_cov)
print(inv_cov)

[[ 1.97600296e+08 -2.43795484e+07  2.39482708e+06]
 [-2.43795484e+07  2.07140427e+08 -2.43704583e+07]
 [ 2.39482708e+06 -2.43704583e+07  9.30896561e+07]]


In [22]:
# Create constant C arrays for model (mu and inverse covariance)
c_mu = c_writer.create_array(model_mu, 'float', c_mean_name)
c_inv_cov = c_writer.create_array(inv_cov, 'float', c_inv_cov_name)

In [23]:
# Construct header file
header_str = c_writer.create_header(c_mu + '\n' + c_inv_cov, c_model_name)
print(header_str)

#ifndef MD_MODEL-STEADY_H
#define MD_MODEL-STEADY_H

const unsigned int model_mu_dim1 = 3;

const float model_mu[3] = {
    0.000679659305993691, 0.0006613785488958986, 0.0008088753943217653
};

const unsigned int model_inv_cov_dim1 = 3;
const unsigned int model_inv_cov_dim2 = 3;

const float model_inv_cov[3][3] = {
    197600295.65531713, -24379548.446011353, 2394827.077276792, 
    -24379548.446011353, 207140426.82222793, -24370458.342991024, 
    2394827.0772767914, -24370458.342991017, 93089656.12940083
};

#endif //MD_MODEL-STEADY_H


In [24]:
# Save C header file
with open(join(models_path, c_model_name) + '.h', 'w') as file:
    file.write(header_str)

Save Samples for Testing
---
Convert raw sample to constant C array for testing.

In [25]:
# Saved Numpy test samples file location
sample_file_path = 'test_samples'
sample_file_name = 'normal_anomaly_samples'  # Will be given .npz suffix
c_normal_sample_name = 'normal_sample'       # Will be given .h suffix for file
c_anomaly_sample_name = 'anomaly_sample'     # Will be given .h suffix for file

In [26]:
# Load test samples
with np.load(join(sample_file_path, sample_file_name) + '.npz') as data:
    normal_sample = data['normal_sample']
    anomaly_sample = data['anomaly_sample']
print(normal_sample.shape)
print(anomaly_sample.shape)
print(normal_sample[:5])
print(anomaly_sample[:5])

(200, 3)
(200, 3)
[[0.056425 0.041053 0.992775]
 [0.055632 0.042395 0.989115]
 [0.057096 0.041175 0.990152]
 [0.05673  0.041175 0.988749]
 [0.053436 0.042212 0.991677]]
[[ 0.015625  0.23584   0.975098]
 [-0.040527  0.192383  0.966797]
 [ 0.006836  0.212891  0.984375]
 [-0.027344  0.196289  0.973633]
 [-0.008789  0.212891  0.986816]]


In [27]:
# Create C array out of normal sample
c_sample = c_writer.create_array(normal_sample, 'float', c_normal_sample_name)
header_str = c_writer.create_header(c_sample, c_normal_sample_name)
#print(header_str)

In [28]:
# Save C header file with normal sample
with open(join(sample_file_path, c_normal_sample_name) + '.h', 'w') as file:
    file.write(header_str)

In [29]:
# Create C array out of anomaly sample
c_sample = c_writer.create_array(anomaly_sample, 'float', c_anomaly_sample_name)
header_str = c_writer.create_header(c_sample, c_anomaly_sample_name)
print(header_str)

#ifndef ANOMALY_SAMPLE_H
#define ANOMALY_SAMPLE_H

const unsigned int anomaly_sample_dim1 = 200;
const unsigned int anomaly_sample_dim2 = 3;

const float anomaly_sample[200][3] = {
    0.015625, 0.23584, 0.975098, -0.040527, 0.192383, 0.966797, 0.006836, 
    0.212891, 0.984375, -0.027344, 0.196289, 0.973633, -0.008789, 0.212891, 
    0.986816, -0.004883, 0.216309, 0.973633, -0.031738, 0.192383, 0.977539, 
    0.021484, 0.233398, 0.98877, -0.033691, 0.188477, 0.97168, 0.005371, 
    0.20752, 0.966309, -0.02002, 0.222168, 0.993164, -0.043457, 0.186035, 
    0.976563, -0.002441, 0.210449, 0.984375, -0.018555, 0.199707, 0.966309, 
    -0.018555, 0.202637, 0.980469, 0.014648, 0.237305, 0.985352, -0.041992, 
    0.182129, 0.97168, 0.005371, 0.217285, 0.980469, -0.021484, 0.193359, 
    0.978027, 0.000977, 0.208984, 0.985352, -0.005371, 0.213867, 0.981445, 
    -0.030273, 0.189453, 0.975098, 0.013184, 0.23291, 0.993652, -0.030273, 
    0.188965, 0.98291, 0.005371, 0.212891, 0.972168, 0.00976

In [30]:
# Save C header file with normal sample
with open(join(sample_file_path, c_anomaly_sample_name) + '.h', 'w') as file:
    file.write(header_str)

Develop C functions
---
We don't have access to an easy/efficient library for calculating the Mahalanobis Distance in C, so we need to write a few functions from scratch. It's easiest to get the algorithms down in Python first so we can compare the output from a few samples to known good algorithms.

In [31]:
# Set some settings to test out our functions
sensor_sample_rate = 200    # Hz
sample_time = 0.64           # Time (sec) length of each sample
max_measurements = int(sample_time * sensor_sample_rate)

In [32]:
# Test extracting features (median absolute deviation) using SciPy
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
sp_test_features = stats.median_abs_deviation(sample)  # Calculate MAD
print(sp_test_features)

[0.000671 0.00061  0.000854]


In [33]:
# Find median in an array (we'll use qsort in C)
# We want to calculate MAD in a C-like fashion to make it easier to port
def median(arr):
    n = len(arr)
    sorted_arr = np.sort(arr)
    if n % 2 == 0:
        median = (sorted_arr[int((n - 1) / 2)] + sorted_arr[int(n / 2)]) / 2.0
    else:
        median = sorted_arr[n / 2]
    return median

In [34]:
# Calculate MAD manually
def calc_mad(arr):
    
    # Find number of elements
    n = len(arr)
    
    # Get median of array
    med = median(arr)

    # Calculate absolute deviation from median for each element
    devs = [0] * n
    for i in range(n):
        devs[i] = abs(arr[i] - med)
    
    # Find median of deviations
    return median(devs)

In [35]:
# Wrapper to extract MAD features for all axis
def extract_mad_features(sample, scale=1.4826):
    mads = []
    
    # Calculate MAD for each axis (note: SciPy scales by 1.4826)
    # https://github.com/scipy/scipy/issues/11090
    for axis in range(sample.shape[1]):
        mads.append(scale * calc_mad(sample[:, axis]))
        
    return mads

In [36]:
# Test MAD algorithm and compare to SciPy answer
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
normal_x = extract_mad_features(sample)
print(normal_x)

[0.000994824599999997, 0.0009043859999999991, 0.0012661404000000316]


In [37]:
# Test: calculate Mahalanobis distance using Numpy
x_minus_mu = normal_x - model_mu
left_term = np.dot(x_minus_mu, inv_cov)
mahal = np.dot(left_term, x_minus_mu.T)
print(mahal)

42.86377903330594


In [38]:
# Calculate the dot product of two vectors
def dot_product(a, b):
    sum = 0
    for i in range(len(a)):
        sum += a[i] * b[i]
        
    return sum

In [39]:
# Do matrix multiplication
def matrix_multiply(a, b):
    
    # Find number of rows and columns
    a_rows = len(a)
    a_cols = len(a[0])
    b_rows = len(b)
    b_cols = len(b[0])
    
    # Check to make sure we can multiply the matrices
    if a_cols != b_rows:
        print('Dimension mismatch')
        return [[]]
    
    # Create return matrix
    prod = [[0 for i in range(b_cols)] for j in range(a_rows)]
    
    # Calculate the dot product for each element in the return matrix
    for i in range(a_rows):
        for j in range(b_cols):
            for k in range(a_cols):
                prod[i][j] += a[i][k] * b[k][j]
    
    return prod

In [40]:
# Calculate Mahalanobis distance the C way (with const inverse covariance)
def mahalanobis(x, mu, inv_cov):
    
    # Subtract each element in X from the mean
    x_minus_mu = [0 for i in range(len(x))]
    for i in range(len(x)):
        x_minus_mu[i] = x[i] - mu[i]
        
    # Compute product of prev term and inverse covariance
    left_term = matrix_multiply([x_minus_mu], inv_cov)
    
    # Transpose difference matrix
    x_minus_mu_t = [[i] for i in x_minus_mu]
    
    # Matrix multiply prev term and difference
    mahal = matrix_multiply(left_term, x_minus_mu_t)
    
    return mahal[0][0]

In [41]:
# Test Mahalanobis function with normal sample
print('Normal Sample')
print('MAD:', normal_x)
print('Mahalanobis distance:', mahalanobis(normal_x, model_mu.tolist(), inv_cov.tolist()))

Normal Sample
MAD: [0.000994824599999997, 0.0009043859999999991, 0.0012661404000000316]
Mahalanobis distance: 42.86377903330594


In [42]:
# Test Mahalanobis function with anomaly sample
print('Anomaly Sample')
sample = anomaly_sample[0:max_measurements]  # Truncate to 128 measurements
anomaly_x = extract_mad_features(sample)
print('MAD:', anomaly_x)
print('Mahalanobis distance:', mahalanobis(anomaly_x, model_mu, inv_cov))

Anomaly Sample
MAD: [0.0253368927, 0.019184102700000007, 0.009049049100000059]
Mahalanobis distance: 168790.3728111164
